Key features in v8:

1. Long Short

Reference
Binance Margin Bracket
https://www.binance.com/en/futures/trading-rules/quarterly/leverage-margin

## Import Library

In [3]:
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import ta

import keras

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

2024-09-01 01:04:33.357776: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
import warnings
warnings.filterwarnings("ignore")

## Defining Functions

### Feature Engineering

In [5]:
def addSMA(data, sma_start: int, sma_end: int, sma_step: int):
    sma_range = range(sma_start, sma_end, sma_step)

    for window in sma_range:
        # SMA/Close
        # relative value of SMA to closing price
        data.loc[:,f"SMA_{window}"] = ta.trend.SMAIndicator(data.Close, window=window).sma_indicator() / data.Close

    for i in range(len(sma_range)):
        for j in range(i+1, len(sma_range)):
            # (SMA_1 - SMA_2) / Close
            data.loc[:,f"SMA_DELTA_{sma_range[i]}_{sma_range[j]}"] = data.loc[:,f"SMA_{sma_range[i]}"] - data.loc[:,f"SMA_{sma_range[j]}"]

    return data

In [6]:
def addEMA(data, ema_start: int, ema_end: int, ema_step: int):
    ema_range = range(ema_start, ema_end, ema_step)

    for window in ema_range:
        # EMA/Close
        # relative value of EMA to closing price
        data.loc[:,f"EMA_{window}"] = ta.trend.EMAIndicator(data.Close, window=window).ema_indicator() / data.Close

    for i in range(len(ema_range)):
        for j in range(i+1, len(ema_range)):
            # (EMA_1 - EMA_2) / Close
            data.loc[:,f"EMA_DELTA_{ema_range[i]}_{ema_range[j]}"] = data.loc[:,f"EMA_{ema_range[i]}"] - data.loc[:,f"EMA_{ema_range[j]}"]

    return data

In [7]:
def feature_generation(
    data,
    sma_start: int = 20,
    sma_end  : int = 100,
    sma_step : int = 20,
    ema_start: int = 20,
    ema_end  : int = 100,
    ema_step : int = 20
    ):
    """
    Input
    --------
    X: pd.DataFrame


    Output
    --------
    X: pd.DataFrame
    """
    close = data.Close

    # SMA
    data = addSMA(data=data, sma_start=sma_start, sma_end=sma_end, sma_step=sma_step)

    # EMA
    data = addEMA(data=data, ema_start=ema_start, ema_end=ema_end, ema_step=ema_step)

    # RSI
    data["RSI"] = ta.momentum.RSIIndicator(data.Close).rsi()
  
    # MACD
    data["MACD"] = ta.trend.MACD(data.Close).macd()

    # ATR
    data["ATR"] = ta.volatility.AverageTrueRange(data.High, data.Low, data.Close).average_true_range()

    # BollingerBands
    upper = ta.volatility.BollingerBands(data.Close).bollinger_hband()
    lower = ta.volatility.BollingerBands(data.Close).bollinger_lband()

    data["BB_upper"] = (upper - close) / close
    data["BB_lower"] = (lower - close) / close
    data["BB_width"] = (upper - lower) / close

    # Garman Klass Volatility
    data["garman_klass_vol"] = ((np.log(data.High)-np.log(data.Low))**2)/2-(2*np.log(2)-1)*((np.log(data.Close)-np.log(data.Open))**2)

    return data

In [8]:
def preprocess(data,
               PCA_components = 8):
  # Feature generation
  data_features = feature_generation(data)

  # Feature
  remove_columns = ["Open",
                    "High",
                    "Low",
                    "Close"]
  features = data_features.drop(remove_columns, axis=1).dropna()

  # normalize
  scaler = StandardScaler()
  features = pd.DataFrame(
    scaler.fit_transform(features),
    index=features.index,
    columns=features.columns)

  # PCA
  pca = PCA(n_components=PCA_components)
  features = pd.DataFrame(
    pca.fit_transform(features),
    index=features.index)
  
  return features

In [9]:
def create_LSTM_input(data, time_steps=10):
    num_samples = data.shape[0] - time_steps + 1
    X = np.array([data[i:i+time_steps] for i in range(num_samples)])
    return X


## Defining Model

In [10]:
def get_model():
    lstm_stacks = 3

    inputs = keras.Input((10, 8)) # Hard-coded timestep and number of features
    outputs = None

    for n in range(lstm_stacks):
        if outputs is None:
            outputs = keras.layers.LSTM(64, return_sequences=True)(inputs)
            outputs = keras.layers.Dropout(0.05)(outputs)
        else:
            if n < lstm_stacks - 1:
                outputs = keras.layers.LSTM(64, return_sequences=True)(outputs)
                outputs = keras.layers.Dropout(0.05)(outputs)
            else:
                outputs = keras.layers.LSTM(64)(outputs)
                outputs = keras.layers.Dropout(0.05)(outputs)
    outputs = keras.layers.Dense(1)(outputs)

    model = keras.Model(inputs, outputs)
    optimizer = keras.optimizers.Adam()
    loss = keras.losses.MeanSquaredError()
    model.compile(optimizer=optimizer,loss=loss)

    return model

## Defining Trading Environment

In [11]:
class Coin:
    def __init__(self, symbol, ohlcv):
        self.symbol = symbol
        self.ohlcv = ohlcv
        self.logret = self.calculate_logret()
        self.trading_history = []  # List to store trading history
        self.volatility = self.calculate_volatility()
        self.features = preprocess(
            pd.concat([
                self.ohlcv,
                self.logret,
                self.volatility],
                axis=1))
        self.y = self.logret.shift(-1)

    def calculate_logret(self):
        return pd.DataFrame(
            np.log((self.ohlcv.Close / self.ohlcv.Close.shift(1)))).rename(columns={"Close": "logret"})

    def calculate_volatility(self):
        return pd.DataFrame(
            self.logret.rolling(window=14).std()).rename(columns={"logret": "vol"})

In [12]:
class Market:
    def __init__(
        self,
        cash = 10000):
        self.coins = {}  # Dictionary to store coins
        self.init_cash = cash
        self.cash = self.init_cash  # Initial cash balance
        self.assets = {}  # Dictionary to store quantity of each asset
        self.prices = pd.DataFrame()

        self.margin_rate = 0.004


    def add_coin(self, coin):
        self.coins[coin.symbol] = coin
        self.assets[coin.symbol] = 0  # Initialize asset quantity to 0

        close = pd.DataFrame(coin.ohlcv.Close).rename(columns={"Close": coin.symbol})
        self.prices = pd.concat([self.prices, close], axis=1)
        
        self.coins[coin.symbol].trading_history = []
        self.coins[coin.symbol].position = "Closed"

        self.coins[coin.symbol].entry_cost = 0

    def reset(self):
        self.cash = self.init_cash
        for symbol in self.coins:
            self.coins[symbol].trading_history = []
            self.assets[symbol] = 0  # Initialize asset quantity to 0
            self.coins[symbol].position = "Closed"
            self.coins[symbol].entry_cost = 0

    def trade(self, symbol, action, quantity, price, time):
        if action == "long":
            self.market_long(symbol, quantity, price, time)

        if action == "short":
            self.market_short(symbol, quantity, price, time)

        elif action == "close":
            if self.coins[symbol].position != "Closed":
                if self.coins[symbol].position == "Long":
                    self.market_long_close(symbol, quantity, price, time)
                if self.coins[symbol].position == "Short":
                    self.market_short_close(symbol, quantity, price, time)

    def market_long(self, symbol, quantity, price, time):        
        available_margin = self.cash
        maintenance_margin = quantity * price * self.margin_rate
        margin_balance = available_margin
        margin_ratio = maintenance_margin / margin_balance

        if margin_ratio < 1:
            self.assets[symbol] += quantity
            self.coins[symbol].trading_history.append(
                {
                    "action": "long",
                    "quantity": quantity,
                    "price": price,
                    "time": time
                })
            self.coins[symbol].entry_cost = quantity * price
            self.coins[symbol].position = "Long"

    def market_long_close(self, symbol, quantity, price, time):
        notional_value = quantity * price

        self.cash += notional_value - self.coins[symbol].entry_cost

        self.assets[symbol] -= quantity
        self.coins[symbol].trading_history.append(
            {
                "action": "close",
                "quantity": quantity,
                "price": price,
                "time": time
            })
        self.coins[symbol].entry_cost = 0
        self.coins[symbol].position = "Closed"

    def market_short(self, symbol, quantity, price, time):
        self.assets[symbol] += quantity
        self.coins[symbol].trading_history.append(
            {
                "action": "short",
                "quantity": quantity,
                "price": price,
                "time": time
            })
        self.coins[symbol].entry_cost = quantity * price
        self.coins[symbol].position = "Short"

    def market_short_close(self, symbol, quantity, price, time):
        notional_value = quantity * price

        self.cash += notional_value - self.coins[symbol].entry_cost

        self.assets[symbol] -= quantity
        self.coins[symbol].trading_history.append(
            {
                "action": "close",
                "quantity": quantity,
                "price": price,
                "time": time
            })
        self.coins[symbol].entry_cost = 0
        self.coins[symbol].position = "Closed"

    def get_margin_ratio(self, time):
        maintenance_margin = 0
        margin_balance = self.cash
        for symbol in self.coins:
            qty = self.assets[symbol]
            price = self.prices.loc[time, symbol]
            entry_cost = self.coins[symbol].entry_cost

            notional_value = qty * price

            maintenance_margin += notional_value * self.margin_rate
            margin_balance += notional_value - entry_cost
        
        margin_ratio = maintenance_margin / margin_balance

        return margin_ratio


## Parameters

This section contains all the parameters.

In [13]:
symbols = [
    "ADAUSDT",
    "BNBUSDT",
    "BTCUSDT",
    "EOSUSDT",
    "ETHUSDT",
    "LTCUSDT",
    "NEOUSDT",
    "QTUMUSDT",
    "XRPUSDT"
    ]

interval = "15m"

## Training the Model

This section outputs training and testing data for the use of the following sections.

### Creating the market for training the model

In [14]:
market_train = Market()
for symbol in symbols:
    data_train = pd.read_csv(f"Binance Data/Training/{interval}/{symbol}.csv", index_col="index")
    data_train = data_train.set_index(pd.DatetimeIndex(pd.to_datetime(data_train.index)))
    coin = Coin(symbol, data_train)
    market_train.add_coin(coin)

In [15]:
market_val = Market()
for symbol in symbols:
    data_val = pd.read_csv(f"Binance Data/Validation/{interval}/{symbol}.csv", index_col="index")
    data_val = data_val.set_index(pd.DatetimeIndex(pd.to_datetime(data_val.index)))
    coin = Coin(symbol, data_val)
    market_val.add_coin(coin)

### Initializing the model

In [24]:
model = get_model()

### Training the model

In [25]:
for symbol in symbols:
    X_train = create_LSTM_input(market_train.coins[symbol].features)[:-1]
    y_train = market_train.coins[symbol].logret[-X_train.shape[0]:]

    X_val = create_LSTM_input(market_val.coins[symbol].features)[:-1]
    y_val = market_val.coins[symbol].logret[-X_val.shape[0]:]
    model.fit(
        X_train, y_train,
        epochs = 50,
        validation_data=(X_val, y_val))

Epoch 1/50
2183/2183 ━━━━━━━━━━━━━━━━━━━━ 38s 15ms/step - loss: 1.7586e-04 - val_loss: 3.3378e-05
Epoch 2/50
2183/2183 ━━━━━━━━━━━━━━━━━━━━ 29s 13ms/step - loss: 5.9276e-05 - val_loss: 3.1998e-05
Epoch 3/50
2183/2183 ━━━━━━━━━━━━━━━━━━━━ 29s 13ms/step - loss: 5.5937e-05 - val_loss: 2.8807e-05
Epoch 4/50
2183/2183 ━━━━━━━━━━━━━━━━━━━━ 24s 11ms/step - loss: 5.4739e-05 - val_loss: 2.8978e-05
Epoch 5/50
2183/2183 ━━━━━━━━━━━━━━━━━━━━ 24s 11ms/step - loss: 5.4614e-05 - val_loss: 2.8272e-05
Epoch 6/50
2183/2183 ━━━━━━━━━━━━━━━━━━━━ 24s 11ms/step - loss: 5.2617e-05 - val_loss: 2.8644e-05
Epoch 7/50
2183/2183 ━━━━━━━━━━━━━━━━━━━━ 24s 11ms/step - loss: 5.2972e-05 - val_loss: 2.8120e-05
Epoch 8/50
2183/2183 ━━━━━━━━━━━━━━━━━━━━ 24s 11ms/step - loss: 5.7761e-05 - val_loss: 2.8628e-05
Epoch 9/50
2183/2183 ━━━━━━━━━━━━━━━━━━━━ 24s 11ms/step - loss: 5.2150e-05 - val_loss: 2.8050e-05
Epoch 10/50
2183/2183 ━━━━━━━━━━━━━━━━━━━━ 24s 11ms/step - loss: 5.4238e-05 - val_loss: 2.8054e-05
Epoch 11/50
2183/21

In [26]:
model.save("LSTM_full.keras")